In [59]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="OU-Advacheck/deberta-v3-base-daigenc-mgt1a")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at OU-Advacheck/deberta-v3-base-daigenc-mgt1a and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [60]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("OU-Advacheck/deberta-v3-base-daigenc-mgt1a")
model = AutoModelForSequenceClassification.from_pretrained("OU-Advacheck/deberta-v3-base-daigenc-mgt1a")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at OU-Advacheck/deberta-v3-base-daigenc-mgt1a and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
texts_to_test = [
"""Taking pictures of objects in our own solar system is much more difficult than taking pictures of distant galaxies. This is because the objects in our solar system are much closer to us, so they appear smaller in the sky and are harder to see clearly. In addition, planets like Venus are covered in clouds, which makes it difficult to get a clear view of the surface.\n\nTo take a picture of Venus or any other planet, we need to use special telescopes that are designed to capture light from distant objects. These telescopes are usually much larger and more powerful than the ones we use to look at stars and galaxies, because they need to be able to see much finer details on the surface of the planet. Even with these powerful telescopes, it can still be difficult to get a clear view of the surface of a planet like Venus, because the clouds and other atmospheric conditions can block our view.\n\nSo, while it may be possible to get some close-up shots of Venus or other planets in our solar system, it is much more difficult than taking pictures of distant galaxies, which are much further away and easier to see clearly."""
]


In [68]:
inputs = tokenizer(texts_to_test, padding=True, truncation=True, return_tensors="pt")

In [69]:
import torch
import numpy as np

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits.numpy()
    # Apply softmax to get normalized probabilities that sum to 1
    probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)

human_prob, ai_prob = probabilities[0]
# Convert to percentages
human_percentage = human_prob * 100
ai_percentage = ai_prob * 100

In [70]:
print("Probabilities (human, machine):", human_prob, ai_prob)

Probabilities (human, machine): 0.6608815 0.33911854


In [73]:
# Simple threshold for AI detection
AI_THRESHOLD = 50.0  # threshold for AI classification

# Calculate percentage difference
percentage_diff = abs(ai_percentage - human_percentage)

# Determine confidence level based on difference
if percentage_diff <= 40:
    confidence = "Low"
elif percentage_diff <= 70:
    confidence = "Medium"
else:
    confidence = "High"

# Classification logic
if ai_percentage >= AI_THRESHOLD:
    label = "AI"
else:
    label = "Human"

if confidence == "Low":
    label = "Uncertain but it is likely to be" + " " + label

In [74]:
print(f"Detection Results")
print(f"{'-'*40}")
print(f"Human Probability: {human_percentage:.1f}%")
print(f"AI Probability: {ai_percentage:.1f}%")
print(f"{'-'*40}")
print(f"Classification: {label}")
print(f"Confidence: {confidence}")

Detection Results
----------------------------------------
Human Probability: 66.1%
AI Probability: 33.9%
----------------------------------------
Classification: Uncertain but it is likely to be Human
Confidence: Low
